# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Cookbook-for-cantera_tools-module" data-toc-modified-id="Cookbook-for-cantera_tools-module-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cookbook for cantera_tools module</a></div><div class="lev2 toc-item"><a href="#running-a-simulation" data-toc-modified-id="running-a-simulation-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>running a simulation</a></div><div class="lev3 toc-item"><a href="#run_simulation-example" data-toc-modified-id="run_simulation-example-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span><code>run_simulation</code> example</a></div><div class="lev3 toc-item"><a href="#find_ignition_delay-example" data-toc-modified-id="find_ignition_delay-example-112"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span><code>find_ignition_delay</code> example</a></div><div class="lev3 toc-item"><a href="#write-your-own-method" data-toc-modified-id="write-your-own-method-113"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>write your own method</a></div><div class="lev2 toc-item"><a href="#reducing-a-mechanism-by-reactions" data-toc-modified-id="reducing-a-mechanism-by-reactions-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>reducing a mechanism by reactions</a></div><div class="lev2 toc-item"><a href="#analyzing-data" data-toc-modified-id="analyzing-data-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>analyzing data</a></div><div class="lev3 toc-item"><a href="#obtaining-reaction-and-species-data" data-toc-modified-id="obtaining-reaction-and-species-data-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>obtaining reaction and species data</a></div><div class="lev3 toc-item"><a href="#finding-stoichiometric-coefficients" data-toc-modified-id="finding-stoichiometric-coefficients-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>finding stoichiometric coefficients</a></div><div class="lev2 toc-item"><a href="#analyzing-data" data-toc-modified-id="analyzing-data-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>analyzing data</a></div><div class="lev3 toc-item"><a href="#finding-stoichiometric-coefficients" data-toc-modified-id="finding-stoichiometric-coefficients-141"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>finding stoichiometric coefficients</a></div>

# Cookbook for cantera_tools module

This notebook describes some of the methods in this package and how they can be used.

In [1]:
import cantera_tools as ctt
import numpy as np
from scipy import integrate
import cantera as ct
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
import importlib
importlib.reload(ctt)

<module 'cantera_tools' from '/home/mark/workspace/tools/cantera_tools.py'>

## running a simulation

Simulations can be run in two ways: 

1. using methods like `run_simulation` or `find_ignition_delay` which create the simulation objects and store the data for you
2. writing the own simulation objects and saving the data in a more flexible form

### `run_simulation` example

In [61]:
model_link = 'cookbook_files/model2.cti'

# creates the cantera Solution object
solution = ctt.create_mechanism(model_link)

# finds initial mole fraction for a fuel-air ratio of 1 with 30%/70% methanol/propane blend
# for non-combustion conditions, this can be replaced by a dictionary of values {'CH3OH': 0.3, 'C3H8':0.7}
mole_fractions = {'H2':5, 'O2':1, 'AR': 0.3}

# set initial conditions of solution in kelvin pascals and mole fractions
conditions = 1000, 10**6, mole_fractions

# store 100 times between 10^-8s and 1s, with an initial point at t=0
times = np.logspace(-8,0,num=100)
times = np.insert(times,0,0)

# run the simulation
outputs = ctt.run_simulation(solution, conditions, times,
                                      condition_type = 'constant-temperature-and-pressure',
                                      output_reactions = False,
                                      output_directional_reactions = True)

# you can combine outputs how you would like with pd.concat
result = pd.concat([outputs['conditions'], outputs['species'], outputs['directional_reactions']], axis = 'columns')

# data can be saved to avoid rerunning the simulation for data analysis (in most cases)
result.to_pickle('cookbook_files/{}.pic'.format('run_simulation_example'))
result.to_csv('cookbook_files/{}.csv'.format('run_simulation_example'))

### `find_ignition_delay` example

In [60]:
model_link = 'cookbook_files/model.cti'

# creates the cantera Solution object
solution = ctt.create_mechanism(model_link)

# finds initial mole fraction for a fuel-air ratio of 1 with 30%/70% methanol/propane blend
# for non-combustion conditions, this can be replaced by a dictionary of values {'CH3OH': 0.3, 'C3H8':0.7}
mole_fractions = ctt.get_initial_mole_fractions(stoich_ratio = 1,
                                               fuel_mole_ratios = [.3,.7],
                                               oxygen_per_fuel_at_stoich_list = [1.5,5],
                                               fuels = ['CH3OH','C3H8'])

# set initial conditions of solution in kelvin pascals and mole fractions
conditions = 750, 10**6, mole_fractions

# run simulation
ig_delay, run_data, _ = ctt.find_ignition_delay(solution, conditions,
                                               output_profile = True,
                                               skip_data = 1000)


# data can be saved to avoid rerunning the simulation for data analysis (in most cases)
run_data.to_pickle('cookbook_files/{}.pic'.format('find_ignition_delay_example'))

### write your own method

If neither of these methods work for you, you can find the source code of either method and modify it to your needs.

## reducing a mechanism by reactions
The modules can create a reduced mechanism given a list of desired reaction strings, using how cantera represents the reaction strings (this can be found by `solution.reaction_equations()`). It will remove any unused species as well.

In [7]:
desired_reactions = ['CH3OH + O2 <=> CH2OH(29) + HO2(12)',
 'C3H8 + O2 <=> C3H7(61) + HO2(12)',
 'C3H8 + O2 <=> C3H7(60) + HO2(12)',
 'CH3OH + OH(10) <=> CH2OH(29) + H2O(11)',
 'C3H8 + OH(10) <=> C3H7(60) + H2O(11)',
 'C3H8 + OH(10) <=> C3H7(61) + H2O(11)',
 'CH3OH + HO2(12) <=> CH2OH(29) + H2O2(13)',
 'C3H8 + HO2(12) <=> C3H7(61) + H2O2(13)',
 'C3H8 + HO2(12) <=> C3H7(60) + H2O2(13)',
 'C3H7(60) + O2 <=> C3H7O2(78)',
 'C3H7(61) + O2 <=> C3H7O2(80)',]

# make the reduced mechanism using the full mechanism `.cti` file.
solution_reduced = ctt.create_mechanism(model_link, kept_reaction_equations=desired_reactions)

# NOTE: this cantera Solution object can now be used like any other

## analyzing data
### obtaining reaction and species data

In [10]:
# this outputs a dataframe of just species
data_species = ctt.find_species(df)

# this outputs a dataframe of just reactions
data_reactions = ctt.find_reactions(df)

# this outputs a dataframe with just reactions involving methanol
reactions_with_methanol = ctt.find_reactions(df,species = 'CH3OH')

### finding stoichiometric coefficients

In [8]:
# this is wrapper function that outputs the stoichiometric coefficient given reaction strings:
ctt.obtain_stoichiometry_of_species(solution,'CH2OH(29)','CH3OH + HO2(12) <=> CH2OH(29) + H2O2(13)')

1.0

In [27]:
# methods can also deal with a list of reactions
reactions_with_species = ctt.find_reactions(df,'CH2OH(29)')
ctt.obtain_stoichiometry_of_species(solution,'CH2OH(29)',reactions_with_species)

Exception: No reactions found for species CH2OH(29)